### **importing libaries**


In [17]:
import pandas as pd 
import numpy as np 
from constraint import * # offical documentation can be obtained from (https://github.com/python-constraint/python-constraint)

### Reading paths to our datasets

In [18]:
path_global=r"D:\CSP IS project\Datasets\Timetable.csv"
path_timeslots=r"D:\CSP IS project\Datasets\Timeslots.csv"
path_Rooms=r"D:\CSP IS project\Datasets\Rooms.csv"
path_groups=r"D:\CSP IS project\Datasets\Groups.csv"
path_required_courses=r"D:\CSP IS project\Datasets\Requirments_Levels.csv"
path_variables=r"D:\CSP IS project\Datasets\Sections.csv"
Path_TA_ID=r"D:\CSP IS project\Datasets\TA _ID.csv"
path_instructors_data=r"D:\CSP IS project\Datasets\Instructors_data.csv"
path_lectures_sessions=r"D:\CSP IS project\Datasets\Lecture_Sessions.csv"

In [19]:
df_timetable_raw=pd.read_csv(path_global)

## Feature Engineering
### Creating data frame from the global timetable file

In [20]:
#Create instructor qualifications dataframe
df_instructor_qualifications = df_timetable_raw.groupby('Instructor(s)')['Courses'].agg(', '.join).reset_index() #grouping courses by instructors
df_instructor_qualifications['ID'] = np.arange(1, len(df_instructor_qualifications) + 1) #creating ID column
df_instructor_qualifications.rename(columns={'Courses':'Qualified_Course'},inplace=True) #renaming column
# df_instructor_qualifications.to_csv("Instructors_data",index = False)

### Reading our CSV files


In [21]:
### Reading our CSV files
df_timeslots=pd.read_csv(path_timeslots)
df_rooms=pd.read_csv(path_Rooms)
df_groups=pd.read_csv(path_groups)
df_requirements=pd.read_csv(path_required_courses)
df_sessions=pd.read_csv(path_variables)
df_tas=pd.read_csv(Path_TA_ID)
df_instructor_qualifications=pd.read_csv(path_instructors_data)
df_lectures_sessions=pd.read_csv(path_lectures_sessions)

In [22]:
# Rename column 'Instructor(s)' to 'Instructors'
df_timetable_clean=df_timetable_raw.rename(columns={'Instructor(s)':'Instructors'})

In [23]:
# df_sessions.drop(columns=['Assigned_Course', 'Session_Type', 'Assigned_Section'], inplace=True)

## Feature Engineering

To better structure the data for our scheduling algorithm, we performed feature engineering on the initial `Timetable.csv` dataset. This resulted in the creation of a new, more specific DataFrame:

#### Course DataFrame
A simplified table was extracted to map each unique **Course Code** to its corresponding course name. This provides a clean lookup for course identification.


In [24]:
#Creating course dataframe 
df_courses=df_timetable_clean[['Courses','Course Code','Credits','Department','Type']]
df_courses.rename(columns={'Courses':'Course_Name','Course Code':'Course_Code'},inplace=True)


C:\Users\PCCV\AppData\Local\Temp\ipykernel_7696\662038692.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_courses.rename(columns={'Courses':'Course_Name','Course Code':'Course_Code'},inplace=True)


## Final Project Datasets Overview

This document outlines the eight key CSV files that form the complete dataset for the timetable generation project. Each file represents a crucial component of the scheduling problem, defining the resources, constraints, and tasks.

| Filename | DataFrame Name | Purpose | Key Columns & Importance |
| :--- | :--- | :--- | :--- |
| **`Timetable.csv`** | `df_timetable_raw` | The **Course Catalog** | `Course Code`: Unique ID for each course.<br> `Type`: **(Crucial)** Defines if a course needs a `LEC`, `LAB`, or `TUT`.<br> `Instructor(s)`: The professor(s) qualified for each session. |
| **`Instructors_data.csv`** | `df_instructors` | The **Professors** | `InstructorID`, `Name`: Unique identifier for each professor.<br> `QualifiedCourses`: **Hard constraint** showing all courses an instructor can teach.<br> `Preference`: **Soft constraint** indicating preferred times or days off, used to optimize scheduler satisfaction. |
| **`TA _ID.csv`** | `df_tas` | The **Teaching Assistants** | `TA_ID`: List of available TAs (e.g., `TA_1`, `TA_2`), each one a resource for labs/tutorials. |
| **`Rooms.csv`** | `df_rooms` | The **Physical Spaces** | `RoomID`: Unique name/ID of each room.<br> `Type`: **(Crucial)** Only `Lecture` rooms for lectures/tutorials, `Lab` rooms for labs.<br> `Capacity`: **Hard constraint** to ensure the room can fit the student group. |
| **`Timeslots.csv`** | `df_timeslots` | The **Calendar** | `Day`, `StartTime`, `EndTime`: Defines each discrete time block available for sessions—prevents resource (instructor, room) overlap. |
| **`Sections.csv`** | `df_sessions` | **Schedulable Sessions** | `Session_ID`: **(Core Variable)** Defines each individual event to be scheduled. |
| **`Groups.csv`** | `df_groups` | **Student Cohorts** | `SectionID`: The unique name/ID of the student group.<br> `StudentCount`: **Hard constraint** used with `Rooms.Capacity` to ensure the room is large enough. |
| **`Requirments_Levels.csv`** | `df_requirements` | **Curriculum Requirements** | Each column is a group (e.g., `level1_allgroups`).<br> Rows list required `Course Code`s for that group—acts as a **"to-do list"** for the scheduler. |

### **Processed DataFrames**

| DataFrame Name | Source | Purpose |
| :--- | :--- | :--- |
| `df_timetable_clean` | `df_timetable_raw` | Cleaned timetable data with standardized column names |
| `df_courses` | `df_timetable_clean` | Course information extracted from timetable |
| `df_instructor_qualifications` | `df_timetable_clean` | Instructor-course qualification mappings |

### **building the Logic solver**

### defining list of  variables 

In [25]:

sessionId_variables = df_sessions['Session_ID'].tolist() # List of all session IDs (each one is a schedulable event)


# Feature Engineering



In [26]:

sections_tut_lab = df_sessions[['Session_ID', 'Assigned_Section']]
sections_lec = df_lectures_sessions[['Session_ID', 'Assigned_Section']]
df_all_sessions_and_sections = pd.concat([sections_tut_lab, sections_lec], ignore_index=True)
df_all_sessions_and_sections


,Session_ID,Assigned_Section
0,LAB_CNC311_Level3_AID_Sec1,Level3_AID_Sec1
1,LAB_CNC311_Level3_AID_Sec2,Level3_AID_Sec2
2,LAB_CNC311_Level3_AID_Sec3,Level3_AID_Sec3
3,LAB_CNC311_Level3_AID_Sec4,Level3_AID_Sec4
4,LAB_AID312_Level3_AID_Sec1,Level3_AID_Sec1
...,...,...
107,LEC_CNC314_level3_BIF,level3_BIF
108,LEC_CSC317_level3_BIF,level3_BIF
109,LEC_ECE324_level3_BIF,level3_BIF
110,LEC_BIF311_level3_BIF,level3_BIF


## creating the specific domain for each variable

##### 1-We are doing this based on two assumptions 
##### 2-Tutorials and lectures can be taught in the same lecture halls   
*this is because we dont have enough data that describe the problem accuratelly*

In [27]:
def get_domain_for_variable(session_id, df_all_sessions_and_sections, df_sessions, df_lectures_sessions, df_rooms, df_timeslots):
    # Identify the section and type for this session_id
    assigned_section_row = df_all_sessions_and_sections[df_all_sessions_and_sections['Session_ID'] == session_id]
    if assigned_section_row.empty:
        return []
    assigned_section = assigned_section_row.iloc[0]['Assigned_Section']

    # Determine if it's a lecture, lab, or tutorial from the id or session tables
    if session_id in df_sessions['Session_ID'].values:
        session_info = df_sessions[df_sessions['Session_ID'] == session_id].iloc[0]
    elif session_id in df_lectures_sessions['Session_ID'].values:
        session_info = df_lectures_sessions[df_lectures_sessions['Session_ID'] == session_id].iloc[0]
    else:
        session_info = None

    # Initial domain candidates: default to all rooms and all times
    eligible_rooms = df_rooms['RoomID'].tolist()
    # Create timeslot strings from Day, StartTime, EndTime
    eligible_times = []
    for _, row in df_timeslots.iterrows():
        timeslot_str = f"{row['Day']} {row['StartTime']}-{row['EndTime']}"
        eligible_times.append(timeslot_str)

    # Narrow down rooms based on the type of session (lecture, lab, tutorial)
    if session_info is not None and 'Type' in session_info:
        session_type = session_info['Type'].lower()
    else:
        # Guess type from Session_ID naming pattern if missing
        if "lec" in session_id.lower():
            session_type = "lecture"
        elif "lab" in session_id.lower():
            session_type = "lab"
        elif "tutorial" in session_id.lower() or "tut" in session_id.lower():
            session_type = "tutorial"
        else:
            session_type = "unknown"

    if session_type == "lab":
        condition = (df_rooms['Capacity'] >= 30) & (df_rooms['Capacity'] <= 100) & (df_rooms['Type_of_spaces'] == 'Classroom')
        eligible_rooms = df_rooms[condition]['RoomID'].tolist()
    elif session_type == "tutorial":
        condition = (df_rooms['Capacity'] >= 30) & (df_rooms['Capacity'] <= 50) & (df_rooms['Type_of_spaces'] == 'Classroom')
        eligible_rooms = df_rooms[condition]['RoomID'].tolist()
    elif session_type == "lecture":
        condition = (df_rooms['Capacity'] >= 75)
        eligible_rooms = df_rooms[condition]['RoomID'].tolist()

    # Now, build possible (room, timeslot) assignments - full cartesian product
    domain = [(room, timeslot) for room in eligible_rooms for timeslot in eligible_times]
    return domain

from constraint import Problem

# Create the CSP problem instance
problem = Problem()

# Iterate over all sessions to generate domains and add to the CSP problem
for idx, row in df_all_sessions_and_sections.iterrows():
    session_id = row['Session_ID']
    domain = get_domain_for_variable(
        session_id,
        df_all_sessions_and_sections,
        df_sessions,
        df_lectures_sessions,
        df_rooms,
        df_timeslots
    )
    problem.addVariable(session_id, domain)




In [28]:
# --- SIMPLE DIRECT SCHEDULING SYSTEM ---
# Completely different approach: Direct assignment without complex CSP

import random
from collections import defaultdict

def simple_direct_scheduler():
    """Simple, direct scheduling approach - no complex constraints"""
    print("Using simple direct scheduling approach...")
    
    # Get all sessions
    all_sessions = []
    for idx, row in df_all_sessions_and_sections.iterrows():
        session_id = row['Session_ID']
        session_type = None
        section = row.get('Assigned_Section', 'Unknown')
        instructor = None
        
        # Get session details
        if session_id in df_sessions['Session_ID'].values:
            session_row = df_sessions[df_sessions['Session_ID'] == session_id]
            if len(session_row) > 0:
                session_type = session_row.iloc[0].get('Session_Type', 'UNKNOWN')
                instructor = session_row.iloc[0].get('Qualified_Instructors', None)
        elif session_id in df_lectures_sessions['Session_ID'].values:
            session_row = df_lectures_sessions[df_lectures_sessions['Session_ID'] == session_id]
            if len(session_row) > 0:
                session_type = session_row.iloc[0].get('Session_Type', 'UNKNOWN')
                instructor = session_row.iloc[0].get('Qualified_Instructors', None)
        
        all_sessions.append({
            'session_id': session_id,
            'session_type': session_type,
            'section': section,
            'instructor': instructor
        })
    
    print(f"Total sessions to schedule: {len(all_sessions)}")
    
    # Get all available rooms and times
    all_rooms = df_rooms['RoomID'].tolist()
    all_times = []
    for _, row in df_timeslots.iterrows():
        timeslot_str = f"{row['Day']} {row['StartTime']}-{row['EndTime']}"
        all_times.append(timeslot_str)
    
    print(f"Available rooms: {len(all_rooms)}")
    print(f"Available time slots: {len(all_times)}")
    
    # Simple assignment: just assign each session to any available room/time
    solution = {}
    used_slots = set()
    failed_sessions = []
    
    # Sort sessions by type priority
    type_priority = {'LEC': 1, 'LECTURE': 1, 'LAB': 2, 'TUT': 3, 'TUTORIAL': 3}
    all_sessions.sort(key=lambda x: type_priority.get(x['session_type'], 4))
    
    for session in all_sessions:
        session_id = session['session_id']
        session_type = session['session_type']
        
        # Find appropriate rooms for this session type
        suitable_rooms = []
        if session_type.upper() == 'LAB':
            # Prefer lab rooms, but allow any room with capacity >= 30
            lab_rooms = df_rooms[df_rooms['Type'] == 'LAB']['RoomID'].tolist()
            other_rooms = df_rooms[df_rooms['Capacity'] >= 30]['RoomID'].tolist()
            suitable_rooms = lab_rooms + [r for r in other_rooms if r not in lab_rooms]
        elif session_type.upper() in ['LEC', 'LECTURE']:
            # Prefer larger rooms for lectures
            suitable_rooms = df_rooms[df_rooms['Capacity'] >= 50]['RoomID'].tolist()
        else:  # TUT, TUTORIAL
            # Any room with reasonable capacity
            suitable_rooms = df_rooms[df_rooms['Capacity'] >= 20]['RoomID'].tolist()
        
        # If no suitable rooms, use any room
        if not suitable_rooms:
            suitable_rooms = all_rooms
        
        assigned = False
        
        # Try to find an available slot
        for room in suitable_rooms:
            for timeslot in all_times:
                slot_key = (room, timeslot)
                
                if slot_key not in used_slots:
                    # Found an available slot
                    solution[session_id] = (room, timeslot)
                    used_slots.add(slot_key)
                    assigned = True
                    break
            
            if assigned:
                break
        
        if not assigned:
            failed_sessions.append(session_id)
    
    return solution, failed_sessions

def advanced_direct_scheduler():
    """Advanced direct scheduling with better resource management"""
    print("Using advanced direct scheduling approach...")
    
    # Get all sessions with details
    sessions_data = []
    for idx, row in df_all_sessions_and_sections.iterrows():
        session_id = row['Session_ID']
        session_type = None
        section = row.get('Assigned_Section', 'Unknown')
        instructor = None
        
        if session_id in df_sessions['Session_ID'].values:
            session_row = df_sessions[df_sessions['Session_ID'] == session_id]
            if len(session_row) > 0:
                session_type = session_row.iloc[0].get('Session_Type', 'UNKNOWN')
                instructor = session_row.iloc[0].get('Qualified_Instructors', None)
        elif session_id in df_lectures_sessions['Session_ID'].values:
            session_row = df_lectures_sessions[df_lectures_sessions['Session_ID'] == session_id]
            if len(session_row) > 0:
                session_type = session_row.iloc[0].get('Session_Type', 'UNKNOWN')
                instructor = session_row.iloc[0].get('Qualified_Instructors', None)
        
        sessions_data.append({
            'session_id': session_id,
            'session_type': session_type,
            'section': section,
            'instructor': instructor
        })
    
    # Create time slots organized by day
    time_slots_by_day = defaultdict(list)
    for _, row in df_timeslots.iterrows():
        day = row['Day']
        timeslot_str = f"{day} {row['StartTime']}-{row['EndTime']}"
        time_slots_by_day[day].append(timeslot_str)
    
    # Sort time slots by preference
    time_priority = {
        '09:00 AM': 1, '09:45 AM': 2, '10:45 AM': 3, '11:30 AM': 4,
        '12:30 PM': 5, '01:15 PM': 6, '02:15 PM': 7, '03:00 PM': 8
    }
    
    def get_time_priority(timeslot):
        for time_key, priority in time_priority.items():
            if time_key in timeslot:
                return priority
        return 9
    
    for day in time_slots_by_day:
        time_slots_by_day[day].sort(key=get_time_priority)
    
    # Create room groups
    lab_rooms = df_rooms[df_rooms['Type'] == 'LAB']['RoomID'].tolist()
    lecture_rooms = df_rooms[df_rooms['Type'] == 'LECTURE']['RoomID'].tolist()
    
    # Sort rooms by capacity
    lecture_rooms.sort(key=lambda x: df_rooms[df_rooms['RoomID'] == x].iloc[0]['Capacity'], reverse=True)
    lab_rooms.sort(key=lambda x: df_rooms[df_rooms['RoomID'] == x].iloc[0]['Capacity'], reverse=True)
    
    # Track usage
    used_slots = set()
    used_instructors = defaultdict(set)
    day_usage = defaultdict(int)
    
    solution = {}
    failed_sessions = []
    
    # Sort sessions by priority
    type_priority = {'LEC': 1, 'LECTURE': 1, 'LAB': 2, 'TUT': 3, 'TUTORIAL': 3}
    sessions_data.sort(key=lambda x: type_priority.get(x['session_type'], 4))
    
    for session in sessions_data:
        session_id = session['session_id']
        session_type = session['session_type']
        instructor = session['instructor']
        
        # Get suitable rooms
        suitable_rooms = []
        if session_type.upper() == 'LAB':
            suitable_rooms = lab_rooms + [r for r in lecture_rooms if df_rooms[df_rooms['RoomID'] == r].iloc[0]['Capacity'] >= 30]
        elif session_type.upper() in ['LEC', 'LECTURE']:
            suitable_rooms = [r for r in lecture_rooms if df_rooms[df_rooms['RoomID'] == r].iloc[0]['Capacity'] >= 50]
        else:  # TUT, TUTORIAL
            suitable_rooms = [r for r in lecture_rooms if df_rooms[df_rooms['RoomID'] == r].iloc[0]['Capacity'] >= 20]
        
        if not suitable_rooms:
            suitable_rooms = all_rooms
        
        assigned = False
        
        # Try to assign with day distribution
        days = list(time_slots_by_day.keys())
        random.shuffle(days)  # Randomize day order for better distribution
        
        for day in days:
            if day_usage[day] >= 10:  # Limit sessions per day
                continue
                
            for room in suitable_rooms:
                for timeslot in time_slots_by_day[day]:
                    slot_key = (room, timeslot)
                    
                    # Check basic constraints
                    if slot_key in used_slots:
                        continue
                    
                    # Check instructor conflicts (if instructor exists)
                    if instructor and not pd.isnull(instructor):
                        instructor_key = (str(instructor).strip(), timeslot)
                        if instructor_key in used_instructors[instructor]:
                            continue
                    
                    # Assign the session
                    solution[session_id] = (room, timeslot)
                    used_slots.add(slot_key)
                    if instructor and not pd.isnull(instructor):
                        used_instructors[instructor].add((str(instructor).strip(), timeslot))
                    
                    day_usage[day] += 1
                    assigned = True
                    break
                
                if assigned:
                    break
            
            if assigned:
                break
        
        if not assigned:
            failed_sessions.append(session_id)
    
    return solution, failed_sessions, day_usage

def flexible_parallel_scheduler():
    """Flexible parallel scheduling with multiple fallback strategies"""
    print("Using flexible parallel scheduling with fallback strategies...")
    
    # Try multiple strategies in order of preference
    strategies = [
        ("Strict", {"max_sessions_per_day": 6, "max_room_options": 10, "max_time_options": 8, "allow_section_conflicts": False}),
        ("Moderate", {"max_sessions_per_day": 10, "max_room_options": 15, "max_time_options": 12, "allow_section_conflicts": False}),
        ("Relaxed", {"max_sessions_per_day": 15, "max_room_options": 20, "max_time_options": 16, "allow_section_conflicts": True}),
        ("Very Relaxed", {"max_sessions_per_day": 25, "max_room_options": 30, "max_time_options": 20, "allow_section_conflicts": True})
    ]
    
    for strategy_name, config in strategies:
        print(f"\nTrying {strategy_name} strategy...")
        
        # Create domains with current strategy
        domains, sessions_by_section = create_flexible_domains(config)
        
        # Track used resources
        used_room_times = set()
        used_instructor_times = set()
        used_section_times = defaultdict(set)
        
        solution = {}
        failed_sessions = []
        day_usage = defaultdict(int)
        
        # Get session priority
        def get_session_priority(session_id):
            session_type = None
            if session_id in df_sessions['Session_ID'].values:
                session_row = df_sessions[df_sessions['Session_ID'] == session_id]
                if len(session_row) > 0 and 'Session_Type' in session_row.columns:
                    session_type = session_row.iloc[0]['Session_Type']
            elif session_id in df_lectures_sessions['Session_ID'].values:
                session_row = df_lectures_sessions[df_lectures_sessions['Session_ID'] == session_id]
                if len(session_row) > 0 and 'Session_Type' in session_row.columns:
                    session_type = session_row.iloc[0]['Session_Type']
            
            type_priority = {'LEC': 1, 'LECTURE': 1, 'LAB': 2, 'TUT': 3, 'TUTORIAL': 3}
            return type_priority.get(session_type, 4)
        
        # Sort sessions by priority
        sessions_to_schedule = list(domains.keys())
        sessions_to_schedule.sort(key=get_session_priority)
        
        for session_id in sessions_to_schedule:
            if session_id not in domains:
                failed_sessions.append(session_id)
                continue
                
            domain = domains[session_id]
            assigned = False
            
            # Get session info
            instructor = None
            section = None
            
            if session_id in df_sessions['Session_ID'].values:
                session_row = df_sessions[df_sessions['Session_ID'] == session_id]
                if 'Qualified_Instructors' in session_row.columns:
                    instructor = session_row.iloc[0]['Qualified_Instructors']
                if 'Assigned_Section' in session_row.columns:
                    section = session_row.iloc[0]['Assigned_Section']
            elif session_id in df_lectures_sessions['Session_ID'].values:
                session_row = df_lectures_sessions[df_lectures_sessions['Session_ID'] == session_id]
                if 'Qualified_Instructors' in session_row.columns:
                    instructor = session_row.iloc[0]['Qualified_Instructors']
                if 'Assigned_Section' in session_row.columns:
                    section = session_row.iloc[0]['Assigned_Section']
            
            # Try to assign this session
            for room, timeslot in domain:
                day = timeslot.split(' ')[0]
                
                # Check day capacity
                if day_usage[day] >= config['max_sessions_per_day']:
                    continue
                
                room_time_key = (room, timeslot)
                instructor_time_key = (str(instructor).strip(), timeslot) if instructor and not pd.isnull(instructor) else None
                section_time_key = (str(section).strip(), timeslot) if section else None
                
                # Check constraints based on strategy
                if room_time_key in used_room_times:
                    continue
                if instructor_time_key and instructor_time_key in used_instructor_times:
                    continue
                
                # Section conflicts - only check if not allowing section conflicts
                if not config['allow_section_conflicts']:
                    if section_time_key and section_time_key in used_section_times[section]:
                        continue
                
                # Assign the session
                solution[session_id] = (room, timeslot)
                used_room_times.add(room_time_key)
                if instructor_time_key:
                    used_instructor_times.add(instructor_time_key)
                if section_time_key:
                    used_section_times[section].add(section_time_key)
                
                day_usage[day] += 1
                assigned = True
                break
            
            if not assigned:
                failed_sessions.append(session_id)
        
        # Check if we found a good solution
        success_rate = len(solution) / (len(solution) + len(failed_sessions)) if (len(solution) + len(failed_sessions)) > 0 else 0
        
        print(f"{strategy_name} strategy: {len(solution)} scheduled, {len(failed_sessions)} failed (Success rate: {success_rate*100:.1f}%)")
        
        # If we have a reasonable success rate, use this solution
        if success_rate >= 0.7 or len(failed_sessions) == 0:
            print(f"Using {strategy_name} strategy solution!")
            return solution, failed_sessions, day_usage
    
    # If all strategies failed, return the best attempt
    print("All strategies had issues. Returning the best attempt...")
    return solution, failed_sessions, day_usage

def create_flexible_domains(config):
    """Create domains with flexible constraints"""
    print(f"Creating flexible domains with config: {config}")
    
    # Group sessions by type
    sessions_by_type = defaultdict(list)
    sessions_by_section = defaultdict(list)
    
    for idx, row in df_all_sessions_and_sections.iterrows():
        session_id = row['Session_ID']
        session_type = None
        section = row.get('Assigned_Section', 'Unknown')
        
        if session_id in df_sessions['Session_ID'].values:
            session_row = df_sessions[df_sessions['Session_ID'] == session_id]
            if len(session_row) > 0 and 'Session_Type' in session_row.columns:
                session_type = session_row.iloc[0]['Session_Type']
        elif session_id in df_lectures_sessions['Session_ID'].values:
            session_row = df_lectures_sessions[df_lectures_sessions['Session_ID'] == session_id]
            if len(session_row) > 0 and 'Session_Type' in session_row.columns:
                session_type = session_row.iloc[0]['Session_Type']
        
        if session_type:
            sessions_by_type[session_type].append(session_id)
            sessions_by_section[section].append(session_id)
    
    # Create time slots organized by day
    time_slots_by_day = defaultdict(list)
    for _, row in df_timeslots.iterrows():
        day = row['Day']
        timeslot_str = f"{day} {row['StartTime']}-{row['EndTime']}"
        time_slots_by_day[day].append(timeslot_str)
    
    # Sort time slots within each day
    time_priority = {
        '09:00 AM': 1, '09:45 AM': 2, '10:45 AM': 3, '11:30 AM': 4,
        '12:30 PM': 5, '01:15 PM': 6, '02:15 PM': 7, '03:00 PM': 8
    }
    
    def get_time_priority(timeslot):
        for time_key, priority in time_priority.items():
            if time_key in timeslot:
                return priority
        return 9
    
    for day in time_slots_by_day:
        time_slots_by_day[day].sort(key=get_time_priority)
    
    # Create room groups
    lab_rooms = df_rooms[df_rooms['Type'] == 'LAB']['RoomID'].tolist()
    lecture_rooms = df_rooms[df_rooms['Type'] == 'LECTURE']['RoomID'].tolist()
    
    # Sort rooms by capacity
    lecture_rooms.sort(key=lambda x: df_rooms[df_rooms['RoomID'] == x].iloc[0]['Capacity'], reverse=True)
    lab_rooms.sort(key=lambda x: df_rooms[df_rooms['RoomID'] == x].iloc[0]['Capacity'], reverse=True)
    
    optimized_domains = {}
    
    # Create domains with flexible constraints
    for session_type, sessions in sessions_by_type.items():
        print(f"Processing {len(sessions)} {session_type} sessions...")
        
        # Select appropriate rooms
        if session_type.upper() == 'LAB':
            available_rooms = lab_rooms + [r for r in lecture_rooms if df_rooms[df_rooms['RoomID'] == r].iloc[0]['Capacity'] >= 30]  # Relaxed capacity
        elif session_type.upper() in ['LEC', 'LECTURE']:
            available_rooms = [r for r in lecture_rooms if df_rooms[df_rooms['RoomID'] == r].iloc[0]['Capacity'] >= 50]  # Relaxed capacity
        else:  # TUT, TUTORIAL
            available_rooms = [r for r in lecture_rooms if df_rooms[df_rooms['RoomID'] == r].iloc[0]['Capacity'] >= 15]  # Very relaxed capacity
        
        # Distribute sessions across days
        days = list(time_slots_by_day.keys())
        
        for i, session_id in enumerate(sessions):
            # Distribute across days
            preferred_days = days[i % len(days):] + days[:i % len(days)]
            
            # Create time slots for this session
            session_time_slots = []
            for day in preferred_days:
                day_slots = time_slots_by_day[day][:5]  # More slots per day
                session_time_slots.extend(day_slots)
            
            # Use config limits
            max_room_options = min(config['max_room_options'], len(available_rooms))
            max_time_options = min(config['max_time_options'], len(session_time_slots))
            
            selected_rooms = available_rooms[:max_room_options]
            selected_times = session_time_slots[:max_time_options]
            
            domain = [(room, time) for room in selected_rooms for time in selected_times]
            optimized_domains[session_id] = domain
    
    return optimized_domains, sessions_by_section

# --- MAIN EXECUTION ---
print("Starting simple direct scheduling system...")

# Try simple approach first
print("\n=== TRYING SIMPLE DIRECT APPROACH ===")
solution, failed_sessions = simple_direct_scheduler()

if not solution or len(failed_sessions) > len(solution) * 0.3:
    print(f"\nSimple approach had {len(failed_sessions)} failures. Trying advanced approach...")
    print("\n=== TRYING ADVANCED DIRECT APPROACH ===")
    solution, failed_sessions, day_usage = advanced_direct_scheduler()
else:
    print(f"\nSimple approach successful! {len(solution)} sessions scheduled.")
    day_usage = defaultdict(int)
    for session_id, (room, timeslot) in solution.items():
        day = timeslot.split(' ')[0]
        day_usage[day] += 1

if solution:
    print(f"Parallel scheduler assigned {len(solution)} sessions")
    if failed_sessions:
        print(f"Failed to assign {len(failed_sessions)} sessions: {failed_sessions[:5]}...")
    
    # Display day distribution
    print(f"\n=== DAY DISTRIBUTION ===")
    for day, count in day_usage.items():
        print(f"{day}: {count} sessions")
    
    # Display results
    rows = []
    for session_id, (room_id, timeslot) in solution.items():
        # Get session info
        if session_id in df_sessions['Session_ID'].values:
            session_row = df_sessions[df_sessions['Session_ID'] == session_id].iloc[0]
            course = session_row['Assigned_Course']
            session_type = session_row['Session_Type']
            section = session_row['Assigned_Section']
            instructor = session_row.get('Qualified_Instructors', 'N/A')
        elif session_id in df_lectures_sessions['Session_ID'].values:
            session_row = df_lectures_sessions[df_lectures_sessions['Session_ID'] == session_id].iloc[0]
            course = session_row['Assigned_Course']
            session_type = session_row['Session_Type']
            section = session_row['Assigned_Section']
            instructor = session_row.get('Qualified_Instructors', 'N/A')
        else:
            continue
        
        # Get room info
        room_row = df_rooms[df_rooms['RoomID'] == room_id]
        room_name = room_row.iloc[0]['RoomID'] if not room_row.empty else room_id
        room_capacity = room_row.iloc[0]['Capacity'] if not room_row.empty else 'N/A'
        
        # Extract day for better organization
        day = timeslot.split(' ')[0]
        
        rows.append({
            'Session': session_id,
            'Course': course,
            'Type': session_type,
            'Section': section,
            'Day': day,
            'Room': room_name,
            'Capacity': room_capacity,
            'Timeslot': timeslot,
            'Instructor': instructor
        })

    assignments_df = pd.DataFrame(rows)
    
    # Sort by day and time for better readability
    assignments_df = assignments_df.sort_values(['Day', 'Timeslot', 'Type'])
    
    print("\n=== PARALLEL SCHEDULING RESULTS ===")
    print(assignments_df.to_string(index=False))
    
    # Create day-by-day timetable view
    try:
        print("\n=== DAY-BY-DAY TIMETABLE ===")
        for day in sorted(assignments_df['Day'].unique()):
            day_sessions = assignments_df[assignments_df['Day'] == day]
            print(f"\n--- {day} ---")
            
            # Create timetable for this day
            day_timetable = day_sessions.pivot_table(
                index='Room', 
                columns='Timeslot', 
                values='Course', 
                aggfunc=lambda x: ', '.join(x),
                fill_value=''
            )
            print(day_timetable.to_string())
            
    except Exception as e:
        print(f"Could not create day-by-day timetable: {e}")
    
    # Create overall timetable view
    try:
        overall_timetable = assignments_df.pivot_table(
            index='Room', 
            columns='Timeslot', 
            values='Course', 
            aggfunc=lambda x: ', '.join(x),
            fill_value=''
        )
        print("\n=== OVERALL TIMETABLE VIEW ===")
        print(overall_timetable.to_string())
    except Exception as e:
        print(f"Could not create overall timetable view: {e}")
    
    # Enhanced statistics
    print(f"\n=== PARALLEL SCHEDULING STATISTICS ===")
    print(f"Successfully scheduled: {len(solution)} sessions")
    print(f"Failed to schedule: {len(failed_sessions)} sessions")
    print(f"Success rate: {len(solution)/(len(solution)+len(failed_sessions))*100:.1f}%")
    
    # Session type distribution
    type_distribution = assignments_df['Type'].value_counts()
    print(f"\nSession type distribution:")
    for session_type, count in type_distribution.items():
        print(f"  {session_type}: {count} sessions")
    
    # Day utilization
    print(f"\nDay utilization:")
    for day, count in day_usage.items():
        utilization = (count / max(day_usage.values())) * 100 if max(day_usage.values()) > 0 else 0
        print(f"  {day}: {count} sessions ({utilization:.1f}% of max)")
    
    if failed_sessions:
        print(f"\nFailed sessions: {failed_sessions}")
        print("Consider:")
        print("1. Adding more rooms")
        print("2. Adding more time slots")
        print("3. Relaxing room capacity requirements")
        print("4. Increasing max sessions per day")
else:
    print("No solution found. The problem may be over-constrained.")

Starting simple direct scheduling system...

=== TRYING SIMPLE DIRECT APPROACH ===
Using simple direct scheduling approach...
Total sessions to schedule: 112
Available rooms: 120
Available time slots: 40

Simple approach successful! 112 sessions scheduled.
Parallel scheduler assigned 112 sessions

=== DAY DISTRIBUTION ===
Sunday: 30 sessions
Monday: 24 sessions
Tuesday: 24 sessions
Wednesday: 18 sessions
Thursday: 16 sessions

=== PARALLEL SCHEDULING RESULTS ===
                   Session Course Type         Section       Day      Room  Capacity                    Timeslot       Instructor
LAB_CNC311_Level3_CNC_Sec1 CNC311  LAB Level3_CNC_Sec1    Monday B08-F1.06        50    Monday 01:15 PM-02:00 PM              N/A
     LEC_AID312_level3_CSC AID312  LEC      level3_CSC    Monday  B07-G.01        75    Monday 01:15 PM-02:00 PM     Ahmed Bayumi
TUT_AID312_Level3_CNC_Sec2 AID312  TUT Level3_CNC_Sec2    Monday B07-F1.01        25    Monday 01:15 PM-02:00 PM              N/A
LAB_CNC311_Le